In [6]:
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC 
from sklearn.metrics import classification_report

####schema?

In [8]:
cols=['sno','app_no','app_type','inst_type','inst_address','inst_address2','city','district','pincode','minority_inst','womens_inst','state','region','inst_type','program','course','level','specialization','intake','approval']

####missing values

In [19]:
df = pd.read_csv('new_institutes.csv')

client = MongoClient('mongodb://localhost:27017/')
db = client['institutes_db']
collection = db['institutes']


missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values)


df['state'] = df['state'].fillna(df['state'].mode()[0])
df['region'] = df['region'].fillna(df['region'].mode()[0])
df['program'] = df['program'].fillna(df['program'].mode()[0])

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x96 in position 33567: invalid start byte

####query

In [13]:
df['students_enrolled'] = df['students_enrolled'].fillna(df['students_enrolled'].median())


records = df.to_dict(orient='records')
collection.insert_many(records)


app = [
    {
        "$group": {
            "_id": "$state",  
            "total_applications": { "$sum": 1 }  
        }
    },
    {
        "$sort": { "total_applications": -1 } 
    }
]
result = collection.aggregate(app)

print("\nApplications per state:")
for state in result:
    print(f"{state['_id']}: {state['total_applications']} applications")


NameError: name 'df' is not defined

####svm and relationship

In [14]:
df_encoded = pd.get_dummies(df, columns=['state', 'region', 'program'])


df_encoded['approval_status'] = df_encoded['approval_status'].apply(lambda x: 1 if x == 'Approved' else 0)


X = df_encoded.drop(columns=['approval_status'])
y = df_encoded['approval_status']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_test)


svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)


print("\nNaive Bayes Classification Report:")
print(classification_report(y_test, y_pred_nb))

print("\nLinear SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))


plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='region', hue='program')

plt.title('Relationship Between Region and Program')
plt.xlabel('Region')
plt.ylabel('Number of Institutes')
plt.xticks(rotation=45)
plt.show()


NameError: name 'df' is not defined